In [6]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup  
from selenium import webdriver

In [2]:
URL = 'https://en.hkp.com.hk/find-property/#list'   # flats available
driver = webdriver.Chrome('./chromedriver')
driver.get(URL)
subhtml = driver.page_source
soup = BeautifulSoup(subhtml, "html.parser")

bedrooms = []
living = []
price = []
e_ratio = []
age = []
area = []

n=0
while n<101:   
    n += 1
    
# bedrooms
    for i in soup.find_all('div', 'list-bedroom'):
        bedrooms.append(i.get_text())
    
# livingrooms
    for i in soup.find_all('div', 'list-living-room'):
        living.append(i.get_text())
    
# price, just extract discounted sellling price & rent, ignore original prices
    for i in soup.find_all('div', 'price_rent'):
        price.append(i.get_text())
    
# efficiency ratio
    for i in soup.find_all('div', 'list-real-use'):
        e_ratio.append(i.get_text())
    
# property age
    for i in soup.find_all('div', 'list-year'):
        age.append(i.get_text())
        
# gross area & gross price per sq.ft.
    for i in soup.find_all('span', 'price-s'):
        area.append(i.get_text())
    
    ulElements = driver.find_elements_by_tag_name("ul")
    for name in ulElements:
        if(name.get_attribute("href") is not None and "javascript:void" in name.get_attribute("href")):
            name.click()
            break

In [3]:
len(bedrooms), len(living), len(price), len(e_ratio), len(age), len(area)

(1818, 1818, 1818, 1818, 1818, 3636)

In [4]:
saleable = area[::2]
gross = area[1::2]
len(saleable), len(gross)

(1818, 1818)

In [7]:
table = pd.DataFrame(np.column_stack([bedrooms, living, price, e_ratio, age, saleable, gross]), 
             columns=['Bedrooms', 'Living_Rooms', 'Price_Data', 'E_Ratio%', 'Age', 'Saleable', 'Gross'])
table.head()

,Bedrooms,Living_Rooms,Price_Data,E_Ratio%,Age,Saleable,Gross
0,4Rm(s),2LR(s),"Price $50.00MRent $85,000",75%(% Efficiency Ratio),9Year(s),"\nSaleable: 1,762' \n\n ...","\nGross: 2,342' \n\n ( ..."
1,5Rm(s),2LR(s),"$98.00M Price $95.00M$138,000 Rent $135,000",68%(% Efficiency Ratio),19Year(s),"\nSaleable: 2,237' \n\n ...","\nGross: 3,305' \n\n ( ..."
2,3Rm(s),2LR(s),"Price $38.00MRent $56,000",- %(% Efficiency Ratio),63Year(s),"\nSaleable: 1,485' \n\n ...",\nGross: - ' \n\n\n\n
3,3Rm(s),2LR(s),$23.80M Price $23.00M,93%(% Efficiency Ratio),38Year(s),"\nSaleable: 1,211' \n\n ...","\nGross: 1,308' \n\n ( ..."
4,3Rm(s),1LR(s),Price $42.00M,78%(% Efficiency Ratio),7Year(s),"\nSaleable: 1,236' \n\n ...","\nGross: 1,591' \n\n ( ..."


In [8]:
# extracting text data using regex
table['Price'] = table["Price_Data"].str.extract('(Price\s\$\d+\.\d+M)')
table['Price'] = table['Price'].str.extract('\$(.*?)M')
table['Rent'] = table["Price_Data"].str.extract('(Rent\s\$\d+\,\d+)')
table['Rent'] = table['Rent'].str.extract('\$(\d+\,\d+)')
table['Rent'] = table['Rent'].str.replace(',', '')
table["E_Ratio%"] = table["E_Ratio%"].str.extract('(\d+)')
table["Bedrooms"] = table["Bedrooms"].str.extract('(\d)')
table["Living_Rooms"] = table["Living_Rooms"].str.extract('(\d)')
table["Age"] = table["Age"].str.extract('(\d*)')
table['Saleable_Area'] =  table['Saleable'].str.extract('\\xa0(\d+?.*?\d+)')
table['Saleable_Area'] = table['Saleable_Area'].str.replace(',', '')
table['Saleable_Price_per_sqft'] = table['Saleable'].str.extract('\$(\d+\,\d+)')
table['Saleable_Price_per_sqft'] = table['Saleable_Price_per_sqft'].str.replace(',', '')
table['Gross_Area'] =  table['Saleable'].str.extract('\\xa0(\d+?.*?\d+)')
table['Gross_Area'] = table['Gross_Area'].str.replace(',', '')
table['Gross_Price_per_sqft'] = table['Gross'].str.extract('\$(\d+\,\d+)')
table['Gross_Price_per_sqft'] = table['Gross_Price_per_sqft'].str.replace(',', '')

# rename column for clear explanation
table = table.rename(columns = {'Price':'Price_M'})

# drop unnecessary columns
table = table.drop(['Price_Data', 'Saleable', 'Gross'], axis=1)

table.head()

,Bedrooms,Living_Rooms,E_Ratio%,Age,Price_M,Rent,Saleable_Area,Saleable_Price_per_sqft,Gross_Area,Gross_Price_per_sqft
0,4,2,75,9,50.00,85000,1762,28377,1762,21349
1,5,2,68,19,95.00,135000,2237,42468,2237,28744
2,3,2,NaN,63,38.00,56000,1485,25589,1485,NaN
3,3,2,93,38,23.00,NaN,1211,18993,1211,17584
4,3,1,78,7,42.00,NaN,1236,33981,1236,26398


In [9]:
table.dtypes

Bedrooms                   object
Living_Rooms               object
E_Ratio%                   object
Age                        object
Price_M                    object
Rent                       object
Saleable_Area              object
Saleable_Price_per_sqft    object
Gross_Area                 object
Gross_Price_per_sqft       object
dtype: object

In [10]:
table = table.apply(pd.to_numeric)
table.dtypes

Bedrooms                     int64
Living_Rooms                 int64
E_Ratio%                   float64
Age                          int64
Price_M                    float64
Rent                       float64
Saleable_Area              float64
Saleable_Price_per_sqft    float64
Gross_Area                 float64
Gross_Price_per_sqft       float64
dtype: object

In [11]:
table.head()

,Bedrooms,Living_Rooms,E_Ratio%,Age,Price_M,Rent,Saleable_Area,Saleable_Price_per_sqft,Gross_Area,Gross_Price_per_sqft
0,4,2,75.0,9,50.0,85000.0,1762.0,28377.0,1762.0,21349.0
1,5,2,68.0,19,95.0,135000.0,2237.0,42468.0,2237.0,28744.0
2,3,2,NaN,63,38.0,56000.0,1485.0,25589.0,1485.0,NaN
3,3,2,93.0,38,23.0,NaN,1211.0,18993.0,1211.0,17584.0
4,3,1,78.0,7,42.0,NaN,1236.0,33981.0,1236.0,26398.0


Unfortunately, the problem of javascript button was found. The above scraped data is just looping for 101 times for the same webpage. 